In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from simpledbf import Dbf5
import datetime as date 

In [3]:
#funcion para calcular outliers
def outliers_obt(data, columna,cuartial1,cuartil2,valoriqr=1.5):
    ##calculamos los cuartiles 
    Q1 = data[columna].quantile(float(cuartial1))
    #print('Primer Cuartile', Q1)
    Q3 = data[columna].quantile(float(cuartil2))
    #print('Tercer Cuartile',Q3)
    IQR = Q3 - Q1
    #print('Rango intercuartile', IQR)

    ##calculamos los bigotes superior e inferior
    BI = (Q1 - valoriqr * IQR)
    #print('bigote Inferior \n', BI)
    BS = (Q3 + valoriqr * IQR)
    #print('bigote superior \n', BS)

    ##obtenemos una nueva tabla sin los outliers
    ubi_sin_out = data[(data[columna] >= BI) & (data[columna] <= BS)]
    return ubi_sin_out

In [4]:
directorio_parquet = ('datasets/yellow_tripdata_2018-01.parquet')

In [5]:
df = pd.read_parquet(directorio_parquet)
df.head(2)

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/yellow_tripdata_2018-01.parquet'

In [ ]:
df=outliers_obt(df,'total_amount','0.25','0.75',valoriqr=4.5)

In [ ]:
df.shape[0]

8485211

In [ ]:
df=outliers_obt(df,'improvement_surcharge','0.25','0.75',valoriqr=4.5)

In [ ]:
df.shape[0]

8479051

In [ ]:
df =outliers_obt(df,'tip_amount','0.25','0.75',valoriqr=2)

In [ ]:
df.shape[0]

8292691

In [ ]:
df=outliers_obt(df,'mta_tax','0.25','0.75',valoriqr=1.5)

In [ ]:
df.shape[0]

8282577

In [ ]:
df=outliers_obt(df,'extra','0.25','0.75',valoriqr=1.5)

In [ ]:
df.shape[0]

8282549

In [ ]:
df=outliers_obt(df,'fare_amount','0.25','0.75',valoriqr=4.5)

In [ ]:
df.shape[0]

8246764

In [ ]:
df=outliers_obt(df,'trip_distance','0.25','0.75',valoriqr=3)

In [ ]:
df.shape[0]

7955228

In [ ]:
df=outliers_obt(df,'tpep_dropoff_datetime','0.25','0.75',valoriqr=1.5)

In [ ]:
df.shape[0]

7955188

In [ ]:
df_changes = df.copy()

In [ ]:
df_changes['congestion_surcharge'].fillna(0.0, inplace=True)
df_changes['airport_fee'].fillna(0.0, inplace=True)

In [ ]:
df_changes['total_amount_1'] = df_changes.total_amount - df_changes.tolls_amount - df_changes.congestion_surcharge - df_changes.airport_fee
df_changes.drop(['congestion_surcharge','airport_fee'], axis=1, inplace=True)
df_changes['Travel_time'] = df_changes.tpep_dropoff_datetime - df_changes.tpep_pickup_datetime
df_changes['LocationID']=df_changes['PULocationID']

In [ ]:
directorio_Location = ('./datasets/taxi+_zone_lookup.csv')

In [ ]:
df_Taxi_zone = pd.read_csv(directorio_Location)

In [ ]:
df_Taxi_zone['Zone'].fillna('Not specified',inplace=True)
df_Taxi_zone['service_zone'].fillna('Not specified',inplace=True)
df_changes= df_changes.merge(df_Taxi_zone, how='left', on='LocationID')

In [ ]:
df_changes.reset_index(inplace=True)
df_changes.rename(columns={'index':'IdTaxis_2018'}, inplace=True)
df_changes.rename(columns={ 'VendorID':'IdVendor','RatecodeID':'IdRatecode','PULocationID':'IdPULocation','DOLocationID':'IdDOLocation',
                            'payment_type':'IdPayment_type','LocationID':'IdLocation'},inplace=True)

In [ ]:
df_changes.columns

Index(['IdTaxis_2018', 'IdVendor', 'tpep_pickup_datetime',
       'tpep_dropoff_datetime', 'passenger_count', 'trip_distance',
       'IdRatecode', 'store_and_fwd_flag', 'IdPULocation', 'IdDOLocation',
       'IdPayment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount',
       'total_amount_1', 'Travel_time', 'IdLocation', 'Borough', 'Zone',
       'service_zone'],
      dtype='object')

In [ ]:
df_changes.drop(df_changes[df_changes['tpep_pickup_datetime'] > '2018-02-01'].index, inplace = True)

In [ ]:
taxi_trip_2018 = df_changes[[   'IdTaxis_2018','IdVendor','tpep_pickup_datetime','tpep_dropoff_datetime','Travel_time','IdRatecode','IdPULocation'
                                ,'IdDOLocation','IdPayment_type','Borough','fare_amount','extra','mta_tax','tip_amount','improvement_surcharge','total_amount_1']]

def algo1(params):
    try:
        return Borough.IdBorough[Borough.Borough== params].iloc[0]
    except:
        return 7

s = taxi_trip_2018.Borough.head(5).apply(algo1)

s

taxi_trip_2018.rename(columns={'Borough':'IdBorough'}, inplace=True)

In [ ]:
vendor={    1:'Creative Mobile Technologies, LLC',
            2:'VeriFone Inc'}

vendor = pd.DataFrame([[key, vendor[key]] for key in vendor.keys()], columns=['IdVendor', 'Name_vendor'])

In [ ]:
Ratecode={  1:'Tarifa estándar',
            2:'jfk',
            3:'nuevaark',
            4:'nassau o westchester',
            5:'tarifa negociada',
            6:'paseo en grupo'}

Ratecode = pd.DataFrame([[key, Ratecode[key]] for key in Ratecode.keys()], columns=['IdRatecode', 'Name_ratecode'])

In [ ]:
payment={   1:'tarjeta de crédito',
            2:'efectivo',
            3:'sin cargo',
            4:'disputa',
            5:'desconocido',
            6:'viaje anulado'}

payment = pd.DataFrame([[key, payment[key]] for key in payment.keys()], columns=['IdPayment_type', 'Payment_type'])

In [ ]:
Borough_dic = [ {'IdBorough':1,'Borough':'EWR','Latitude':40.6895314,'Longitude':-74.17446239999998},
                {'IdBorough':2,'Borough':'Queens','Latitude':40.742054,'Longitude':-73.769417},
                {'IdBorough':3,'Borough':'Bronx','Latitude':40.837048,'Longitude':-73.865433},
                {'IdBorough':4,'Borough':'Manhattan','Latitude':40.776676,'Longitude':-73.971321},
                {'IdBorough':5,'Borough':'Staten Island','Latitude':40.579021,'Longitude':-74.151535},
                {'IdBorough':6,'Borough':'Brooklyn','Latitude':40.650002,'Longitude':-73.949997},
                {'IdBorough':7,'Borough':'Unknown','Latitude':0.0,'Longitude':0.0}]

Borough = pd.DataFrame(Borough_dic)

In [ ]:
def algo(params):
    try:
        return Borough.IdBorough[Borough.Borough== params].iloc[0]
    except:
        return 7

In [ ]:
df_Taxi_zone.Borough = df_Taxi_zone.Borough.apply(algo)
df_Taxi_zone.rename(columns={'Borough':'IdBorough'}, inplace=True)
df_Taxi_zone.drop('service_zone',axis=1 ,inplace=True)
Location = df_Taxi_zone.copy()

In [ ]:
df_Taxi_zone

,LocationID,IdBorough,Zone
0,1,1,Newark Airport
1,2,2,Jamaica Bay
2,3,3,Allerton/Pelham Gardens
3,4,4,Alphabet City
4,5,5,Arden Heights
...,...,...,...
260,261,4,World Trade Center
261,262,4,Yorkville East
262,263,4,Yorkville West
263,264,7,NV


In [ ]:
Location

,LocationID,IdBorough,Zone
0,1,1,Newark Airport
1,2,2,Jamaica Bay
2,3,3,Allerton/Pelham Gardens
3,4,4,Alphabet City
4,5,5,Arden Heights
...,...,...,...
260,261,4,World Trade Center
261,262,4,Yorkville East
262,263,4,Yorkville West
263,264,7,NV


In [ ]:
directorio_calendar=('./datasets/calendar.csv')

In [ ]:
calendar = pd.read_csv(directorio_calendar, sep=';')
calendar

,id,hour,date,day,week,year
0,1,00:00:00,1/1/2018,1,1,2018
1,2,01:00:00,1/1/2018,1,1,2018
2,3,02:00:00,1/1/2018,1,1,2018
3,4,03:00:00,1/1/2018,1,1,2018
4,5,04:00:00,1/1/2018,1,1,2018
...,...,...,...,...,...,...
739,740,19:00:00,31/1/2018,31,5,2018
740,741,20:00:00,31/1/2018,31,5,2018
741,742,21:00:00,31/1/2018,31,5,2018
742,743,22:00:00,31/1/2018,31,5,2018


In [ ]:
calendar["Date"] = pd.to_datetime(calendar["date"])
calendar.rename(columns={'id':'IdCalendar'}, inplace=True)
calendar = calendar.reindex(columns=['IdCalendar','Date','year','week','day','hour'])

In [ ]:
lista_table=[Borough, Location,Ratecode,payment,vendor, calendar, taxi_trip_2018]

# ***MACHIN LEARNING***

sns.pairplot(df)